In [36]:
import pandas as pd
from math import sqrt
import numpy as np
from  google.colab import drive

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
movies_df = pd.read_csv('/content/drive/MyDrive/fods_assignment4/fods_assignment4/movies.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/fods_assignment4/fods_assignment4/ratings.csv')
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [39]:
userInput = [{'title':'Mortal Kombat (1995)', 'rating':2},
             {'title':'To Die For (1995)', 'rating':4},
             {'title':'The Lego Batman Movie (2017)', 'rating':5},
             {'title':'Pocahontas (1995)', 'rating':3},
             {'title':'Akira (1988)', 'rating':4}]
inputMovies = pd.DataFrame(userInput)
print(inputMovies)

                          title  rating
0          Mortal Kombat (1995)       2
1             To Die For (1995)       4
2  The Lego Batman Movie (2017)       5
3             Pocahontas (1995)       3
4                  Akira (1988)       4


In [40]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies[['movieId','title','rating']]
print(inputMovies)


   movieId                         title  rating
0       44          Mortal Kombat (1995)       2
1       45             To Die For (1995)       4
2       48             Pocahontas (1995)       3
3     1274                  Akira (1988)       4
4   167746  The Lego Batman Movie (2017)       5


In [41]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
print(userSubset.groupby('movieId').count())


         userId  rating  timestamp
movieId                           
44           46      46         46
45           32      32         32
48           68      68         68
1274         39      39         39
167746        7       7          7


In [42]:
userSubsetGroup = userSubset.groupby(['userId'])

def take_5_elem(x):
    return len(x[1])

userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:5])

[(414,        userId  movieId  rating  timestamp
62319     414       44     2.0  961516249
62320     414       45     3.0  961438476
62323     414       48     3.0  961437741
62769     414     1274     4.0  997200022), (182,        userId  movieId  rating   timestamp
26107     182       44     1.5  1055150820
26108     182       45     3.0  1075764933
26301     182     1274     5.0  1054779799), (274,        userId  movieId  rating   timestamp
39244     274       44     4.0  1171759773
39246     274       48     2.0  1171943719
39448     274     1274     4.0  1205707621), (380,        userId  movieId  rating   timestamp
56889     380       44     4.0  1493667678
56891     380       48     3.0  1493423481
58070     380   167746     5.0  1508436102), (387,        userId  movieId  rating   timestamp
59240     387       44     1.0  1182720368
59241     387       45     2.5  1094877493
59453     387     1274     4.5  1122954482)]


<ipython-input-42-b4951a471509>:6: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)


In [43]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:

    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')

    #Get the N for the formula
    nRatings = len(group)

    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]

    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()

    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()


    #Now let's calculate the pearson correlation between two users, so called, x and y manually (check the formula from week 7 slide)
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [44]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())

   similarityIndex  userId
0         0.852803     414
1         0.821995     182
2         0.000000     274
3         0.654654     380
4         0.821995     387


In [45]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

    similarityIndex  userId
5               1.0     469
30              1.0     428
26              1.0     371
24              1.0     316
15              1.0     117


In [46]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
print(topUsersRating.head(100))

    similarityIndex  userId  movieId  rating  timestamp
0               1.0     469        1     4.0  965336888
1               1.0     469        6     3.0  965336673
2               1.0     469       10     2.0  965334356
3               1.0     469       11     3.0  965425831
4               1.0     469       29     4.0  965335401
..              ...     ...      ...     ...        ...
95              1.0     469     1080     5.0  965336888
96              1.0     469     1082     4.0  965333459
97              1.0     469     1086     4.0  965846225
98              1.0     469     1089     5.0  965336673
99              1.0     469     1090     4.0  965846279

[100 rows x 5 columns]


In [47]:
#Multiplies the similarity by the user’s ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
print(topUsersRating.head())

   similarityIndex  userId  movieId  rating  timestamp  weightedRating
0              1.0     469        1     4.0  965336888             4.0
1              1.0     469        6     3.0  965336673             3.0
2              1.0     469       10     2.0  965334356             2.0
3              1.0     469       11     3.0  965425831             3.0
4              1.0     469       29     4.0  965335401             4.0


In [48]:
#Applies a sum to the topUsers after grouping it up by movieId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

         sum_similarityIndex  sum_weightedRating
movieId                                         
1                  13.695477           50.637523
2                   9.873482           31.813728
3                   6.218828           17.393262
5                   4.852803           13.205606
6                   9.829451           36.780654


In [49]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()

#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
print(recommendation_df.head(10))

         weighted average recommendation score  movieId
movieId                                                
1                                     3.697390        1
2                                     3.222139        2
3                                     2.796871        3
5                                     2.721233        5
6                                     3.741883        6
7                                     3.402040        7
8                                     3.000000        8
9                                     1.833333        9
10                                    3.542422       10
11                                    3.684975       11


In [50]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)

recommendation_df_sorted = recommendation_df[recommendation_df['weighted average recommendation score'] > 4.9]

print(recommendation_df_sorted)

         weighted average recommendation score  movieId
movieId                                                
6192                                       5.0     6192
1295                                       5.0     1295
3925                                       5.0     3925
56715                                      5.0    56715
27523                                      5.0    27523
...                                        ...      ...
71033                                      5.0    71033
1238                                       5.0     1238
84273                                      5.0    84273
1217                                       5.0     1217
6442                                       5.0     6442

[81 rows x 2 columns]


In [51]:
recommended_movie = movies_df.loc[movies_df['movieId'].isin(recommendation_df_sorted['movieId'])]

#we don't want to recommend the same movie
recommended_movie = recommended_movie.loc[~recommended_movie.movieId.isin(userSubset['movieId'])]

print(recommended_movie)

      movieId                                  title  \
251       290              Once Were Warriors (1994)   
361       417                       Barcelona (1994)   
835      1096                 Sophie's Choice (1982)   
918      1217                             Ran (1985)   
938      1238                      Local Hero (1983)   
...       ...                                    ...   
9497   170705                Band of Brothers (2001)   
9546   172875               A Detective Story (2003)   
9550   173145  War for the Planet of the Apes (2017)   
9634   179135                  Blue Planet II (2017)   
9688   184245                De platte jungle (1978)   

                             genres  
251                     Crime|Drama  
361                  Comedy|Romance  
835                           Drama  
918                       Drama|War  
938                          Comedy  
...                             ...  
9497               Action|Drama|War  
9546               Anim